## Use sensors on the IoT-LAB M3 board

The IoT-LAB M3 board provides several sensors:
- lps331ap: an temperature and atmospheric pressure sensor
- l3g4200d: a gyroscope
- lsm303dlhc: an accelerometer and magnetometer
- isl29020: a light sensor

In this exercise, we propose to read and display the values of the lps331ap and lsm303dlhc sensors using RIOT available high-level drivers.

The lps331ap values will be read and displayed from the main thread every 2 seconds and the lsm303dlhc values will be read and displayed from a separate thread every 500ms.

### Start an experiment on IoT-LAB

Choose your site (grenoble|lille|strasbourg):

In [ ]:
%env SITE=grenoble

In [ ]:
!iotlab-experiment submit -n "riot-sensors" -d 20 -l 1,archi=m3:at86rf231+site=$SITE

In [ ]:
!iotlab-experiment wait

In [ ]:
!iotlab-experiment get --nodes

### Read the lps331ap sensor

To read the lps331ap sensor, use the lps331ap module driver.

1. In the application `Makefile`, add the lps331ap module to the build:

```Makefile
USEMODULE += lps331ap
```

2. In the `main.c`, add the necessary includes for this driver:

```c
#include "lpsxxx.h"
#include "lpsxxx_params.h"
```

**Note:** The files, device descriptor type and function names start with `lpsxxx` because this driver is generic and also works with the lps25hb and lps22hb variants.

3. Declare the device descriptor variable:

```c
static lpsxxx_t lpsxxx;
```

4. In the main function, initialize the driver:

```c
lpsxxx_init(&lpsxxx, &lpsxxx_params[0]);
```

5. Read the temperature and pressure values every 2 seconds in an endless loop before the main function returns:

```c
    while (1) {
        uint16_t pres = 0;
        int16_t temp = 0;
        lpsxxx_read_temp(&lpsxxx, &temp);
        lpsxxx_read_pres(&lpsxxx, &pres);
        printf("Pressure: %uhPa, Temperature: %u.%u°C\n",
               pres, (temp / 100), (temp % 100));
        xtimer_sleep(2);
    }
```

6. Build and flash the application:

In [ ]:
!make IOTLAB_NODE=auto-ssh flash

Open a Jupyter terminal (use `File > New > Terminal`) and connect to the serial port of the IoT-LAB M3 and observe the value displayed every 2 seconds:

```sh
make IOTLAB_NODE=auto-ssh -C riot/basics/sensors term
```

Keep the terminal open until you complete the next section.

### Read the lsm303dlhc sensor

To read the lsm303dlhc sensor, use the lsm303dlhc module driver. In this part of the exercise, the driver is read from a separate background thread.

1. In the application `Makefile`, add the lsm303dlhc module to the build:

```Makefile
USEMODULE += lsm303dlhc
```

2. In the `main.c`, add the necessary includes for this driver:

```c
#include "lsm303dlhc.h"
#include "lsm303dlhc_params.h"
```

3. Declare the device descriptor variable:

```c
static lsm303dlhc_t lsm303dlhc;
```

4. In the main function, initialize the driver:

```c
lsm303dlhc_init(&lsm303dlhc, lsm303dlhc_params);
```

5. Read the accelerometer and magnetometer values every 500ms in an endless loop in the thread handler function:

```c
    while (1) {
        lsm303dlhc_3d_data_t mag_value;
        lsm303dlhc_3d_data_t acc_value;
        lsm303dlhc_read_acc(&lsm303dlhc, &acc_value);
        printf("Accelerometer x: %i y: %i z: %i\n",
               acc_value.x_axis, acc_value.y_axis, acc_value.z_axis);
        lsm303dlhc_read_mag(&lsm303dlhc, &mag_value);
        printf("Magnetometer x: %i y: %i z: %i\n",
               mag_value.x_axis, mag_value.y_axis, mag_value.z_axis);
        xtimer_usleep(500 * US_PER_MS);
    }
```

6. Build and flash the application:

In [ ]:
!make IOTLAB_NODE=auto-ssh flash

If you kept the serial output opened in the terminal, you should now get the temperature, pressure, accelerometer and magnetometer values displayed at different rates.

Congratulations, you completed with success all RIOT exercises of this Mooc!

### Free up the resources

Since you finished the training, stop your experiment to free up the experiment nodes:

In [ ]:
!iotlab-experiment stop

The serial link connection through SSH will be closed automatically.